<center><img src="https://storage.googleapis.com/arize-assets/arize-logo-white.jpg" width="200"/></center>

# Credit Risk Assessement in the Financial Servies Industry

Walk through how to use Arize for a credit risk model using an example dataset.

## 📨 Upload Data to Arize

Upload example data to Arize, this example uses the [Python Pandas method](https://docs.arize.com/arize/sending-data-methods/log-directly-via-sdk-api).

In [ ]:
# Install and import dependencies

!pip install -q arize
from arize.pandas.logger import Client, Schema
from arize.utils.types import ModelTypes, Environments, Metrics

import pandas as pd
import numpy as np
import datetime

### 🌐 Upload Data to Arize: Download Data
We'll use a sample Parquet file that represents a model that predicts the risk of lending to an individual based on their credit history. This model includes features such as credit history length, default history, interest rate, etc.

In [ ]:
file_url = "https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/financial_services_credit_scoring.parquet"
df = pd.read_parquet(file_url)
df

In [ ]:
#create prediction timestamp column

current_time = datetime.datetime.now().timestamp()

earlier_time = (
    datetime.datetime.now() - datetime.timedelta(days=30)
).timestamp()

optional_prediction_timestamps = np.linspace(
    earlier_time, current_time, num=df.shape[0]
)

df.insert(1, "prediction_ts", optional_prediction_timestamps.astype(int))
df[["prediction_ts"]].head()

### 🤝 Upload Data to Arize: Create Arize Client
Sign up/login to your Arize account <a href="https://app.arize.com/auth/login">here</a>. Find your <a href="https://docs.arize.com/arize/api-reference/arize.pandas/client">Space and API keys</a>. Copy/paste into the cell below.

In [ ]:
SPACE_KEY = "YOUR_SPACE_KEY"  # update value here with your Space Key
API_KEY = "YOUR_API_KEY"  # update value here with your API key

arize_client = Client(space_key=SPACE_KEY, api_key=API_KEY)

In [ ]:
if SPACE_KEY == "SPACE_KEY" or API_KEY == "API_KEY":
    raise ValueError("❌ NEED TO CHANGE SPACE AND/OR API_KEY")
else:
    print(
        "✅ Import and Setup Arize Client Done! Now we can start using Arize!"
    )

### 📋 Upload Data to Arize: Define Schema
Create your <a href="https://docs.arize.com/arize/sending-data-to-arize/model-schema-reference">model schema</a>. This helps Arize organize your model data such as features, prediction, rank, etc.

In [ ]:
feature_column_names = [
    "person_home_ownership",
    "person_emp_length",
    "loan_intent",
    "loan_grade",
    "loan_amnt",
    "loan_int_rate",
    "loan_status",
    "loan_percent_income",
    "cb_person_default_on_file",
    "cb_person_cred_hist_length",
]

tag_column_names = [
    "person_age",
    "person_income",
]

#Define a Schema() object for Arize to pick up data from the correct columns for logging
schema = Schema(
    timestamp_column_name="prediction_ts",
    prediction_score_column_name="predicted_risk",
    actual_label_column_name="loan_default",
    feature_column_names=feature_column_names,
    tag_column_names=tag_column_names
)

### 🪵 Log Data to Arize
Log the DataFrame using the <a href="https://docs.arize.com/arize/sending-data-to-arize/data-ingestion-methods/sdk-reference/python-sdk/arize.pandas">pandas API</a>.

In [ ]:
response = arize_client.log(
    dataframe=df,
    model_id="credit-risk-financial-services",
    model_version="1.0",
    model_type=ModelTypes.BINARY_CLASSIFICATION,
    validate=True,
    environment=Environments.PRODUCTION,
    schema=schema
)

if response.status_code == 200:
    print(f"✅ You have successfully logged production dataset to Arize")
else:
    print(
        f"Logging failed with response code {response.status_code}, {response.text}"
    )

## 🏃 Follow 'Success!' Link To Arize
Once you've successfully logged your model to Arize, follow the link to setup monitors, uncover problem areas, and more!

<strong>Note</strong>: It might take a few minutes for all the data to index in Arize, if you don't see all the data immedieatly, sit back and relax, data is on it's way!

### 🔍 In Arize: Model Setup
Now that we can see our model data in Arize, let's get our model setup with some basic configurations.
* Navigate to the 'Config' tab and select 'AUC' as the default metric and 'DEAULT'as the positive class
* Since we're leveraging a limited amount of data, let's narrow the production baseline window. Click on 'Configure Baseline' to shorten the baseline window and move it left

AUC summarizes the true positive rate and false positive rate, and helps measure our model's ability to distinguish between customers who likely will or won't default on their loan and their risk.

<image src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/financial_services_credit_setup.png">

### 🔍 In Arize: Monitor Setup

Let's setup an AUC monitor to get alerted when our model deviates from expected behavior.
* Navigate to the 'Monitors' tab and click 'Enable' on the AUC card.
* Scroll through the list of other metrics and monitor types, enable a few that seem interesting!

<image src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/financial_services_credit_monitors.png" width=800px>

### 📈 In Arize: Performance Tracing
Now, let's take a look at the 'Performance Tracing' tab to identify areas to improve and better understand the impact of each feature on our model performance.

* Navigate to the 'Performance Tracing' tab
* Scroll down to the 'Performance Insights' card, click on the worst performing slices

Arize will surface the features that negatively impact your prediction performance the most. Visualize how each component within a given feature impacts your model.

<image src="https://storage.googleapis.com/arize-assets/fixtures/Industry_Use_Case/financial_services_credit_performance.png" width=800px>

## 🚀 Continue Exploring Arize
This tutorial just scratches the surface of what Arize can do. Continue to explore the world of ML Observability with Arize to monitor, troubleshoot, and fine tune your models!

<strong>Recommended Resources:</strong>
* [Arize Community Slack](https://join.slack.com/t/arize-ai/shared_invite/zt-1is2wp3xv-SQgwwszCEeS06Sm1q4xFFw)
* [Arize Documentation](https://docs.arize.com/arize/)
* [ML Observability Course](https://courses.arize.com/)